In [1]:
from kmcluster.core.data import sparse_to_mat
Pt_H1_links = [
    [0, 1, 0.15],
    [0, 2, 0.61],
    [0, 3, 0.39],
    [2, 4, 0.27],
    [2, 6, 0.50],
    [2, 8, 0.66],
    [3, 8, 0.50],
    [5, 7, 0.52],
    [5, 9, 0.66],
    [5, 6, 0.66],
]
H_H1_links = [
    [0, 6, 0.52],
    [1, 2, 0.28],
    [2, 5, 0.22],
    [3, 5, 0.22],
    [7, 8, 0.15],
    [8, 9, 0.14],
]
Pt4H1_rawE = [
    -17.71720725,
    -17.68531409,
    -17.57336808,
    -17.50640668,
    -17.50097929,
    -17.50887522,
    -17.38155630,
    -17.25580137,
    -17.15164472,
    -17.13649884,
]
H1_E = [Pt4H1_rawE[x] - Pt4H1_rawE[0] for x in range(0, len(Pt4H1_rawE))]

Pt4H1_rev = []
for i in range(0, len(Pt_H1_links)):
    Pt4H1_rev.append(
        [
            Pt_H1_links[i][1],
            Pt_H1_links[i][0],
            round(
                (Pt_H1_links[i][2] + H1_E[Pt_H1_links[i][0]]) - H1_E[Pt_H1_links[i][1]],
                2,
            ),
        ]
    )

# raw energies
Pt4H1_rawE = [
    -17.71720725,
    -17.68531409,
    -17.57336808,
    -17.50640668,
    -17.50097929,
    -17.50887522,
    -17.38155630,
    -17.25580137,
    -17.15164472,
    -17.13649884,
]

# relative energies
H1_E = [Pt4H1_rawE[x] - Pt4H1_rawE[0] for x in range(0, len(Pt4H1_rawE))]

# all the forward barriers
Pt_H1_links = [
    [0, 1, 0.15],
    [0, 2, 0.61],
    [0, 3, 0.39],
    [2, 4, 0.27],
    [2, 6, 0.50],
    [2, 8, 0.66],
    [3, 8, 0.55],
    [5, 7, 0.52],
    [5, 9, 0.66],
    [5, 6, 0.66],
    [0, 6, 0.52],
    [1, 2, 0.28],
    [2, 5, 0.22],
    [3, 5, 0.22],
    [7, 8, 0.15],
    [8, 9, 0.14],
]

# calculating the reverse barriers
Pt4H1_rev = []
for i in range(0, len(Pt_H1_links)):
    Pt4H1_rev.append(
        [
            Pt_H1_links[i][1],
            Pt_H1_links[i][0],
            round(
                (Pt_H1_links[i][2] + H1_E[Pt_H1_links[i][0]]) - H1_E[Pt_H1_links[i][1]],
                2,
            ),
        ]
    )

# all barriers
Pt_H1_all = Pt_H1_links + Pt4H1_rev
energies_mat = sparse_to_mat(Pt_H1_all)

In [2]:
energies_mat

array([[0.  , 0.15, 0.61, 0.39, 0.  , 0.  , 0.52, 0.  , 0.  , 0.  ],
       [0.12, 0.  , 0.28, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.47, 0.17, 0.  , 0.  , 0.27, 0.22, 0.5 , 0.  , 0.66, 0.  ],
       [0.18, 0.  , 0.  , 0.  , 0.  , 0.22, 0.  , 0.  , 0.55, 0.  ],
       [0.  , 0.  , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.16, 0.22, 0.  , 0.  , 0.66, 0.52, 0.  , 0.66],
       [0.18, 0.  , 0.31, 0.  , 0.  , 0.53, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.27, 0.  , 0.  , 0.15, 0.  ],
       [0.  , 0.  , 0.24, 0.2 , 0.  , 0.  , 0.  , 0.05, 0.  , 0.14],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.29, 0.  , 0.  , 0.12, 0.  ]])

In [3]:
from kmcluster.core.kmc_minimum import kmc 
from kmcluster.core.transition_conditions import rfkmc
from kmcluster.core.intialize import random_init, boltz
from kmcluster.core.data import sparse_to_mat

T_kelvin = 100
temp_boltz = T_kelvin * 8.617 * 10 ** (-5)
rfkmc_obj = rfkmc(k_b_t=temp_boltz, energy_mat=energies_mat)
time_stop = 0.0001
init_boltz = boltz(energies=H1_E, T=temp_boltz, size=10000)
init_random = random_init(10000, energies_mat.shape[0])

In [4]:
#print mat with 2 decimal places
import numpy as np 
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})


In [5]:

# initialize kmc object
kmc_boltz = kmc(
    time_stop=time_stop,
    energies=energies_mat,
    draw_crit=rfkmc_obj,
    initialization=init_random,
    checkpoint=True,
    sample_frequency=1000,
    ckptprefix="Pt4H2_g_{}_".format(T_kelvin),
    state_dict_file=None, 
    checkpoint_dir="./checkpoints/", # change this to organize runs
    batch_size=1, 
)

# run calcs
# can either run to a certain number of steps or until a certain time
# n_steps=-1 means run until time_stop is reached
kmc_boltz.run(n_steps=-1)
# kmc_boltz.run(n_steps=100)
trajectories = kmc_boltz.trajectories

results mat shape:  (10, 1001)
initialization populations
{0: 958, 1: 990, 2: 1018, 3: 924, 4: 1040, 5: 1032, 6: 988, 7: 992, 8: 1035, 9: 1023}
done initializing....
starting run
> batched step
[958.0, 990.0, 1018.0, 924.0, 1040.0, 1032.0, 988.0, 992.0, 1035.0, 1023.0]
initial probe counts:  10000.0
probe list start:  {0: 958, 1: 990, 2: 1018, 3: 924, 4: 1040, 5: 1032, 6: 988, 7: 992, 8: 1035, 9: 1023}
traj list start:  {0: 958, 1: 990, 2: 1018, 3: 924, 4: 1040, 5: 1032, 6: 988, 7: 992, 8: 1035, 9: 1023}
time traj 0 2.190570547980331e-05
> batched step
probe list start:  {1.0: 1257, 0.0: 1287, 2.0: 881, 8.0: 3, 7.0: 1027}
traj list start:  {1.0: 1384, 0.0: 1288, 2.0: 882, 8.0: 2014, 7.0: 1035}
time traj 0 2.1905843654113974e-05
> batched step
probe list start:  {0.0: 1380, 1.0: 1349, 7.0: 2005, 8.0: 2}
traj list start:  {0.0: 1381, 1.0: 1486, 7.0: 2014, 8.0: 1035}
time traj 0 3.0410076269286034e-05
> batched step
probe list start:  {1.0: 1197, 0.0: 1479, 8.0: 4, 7.0: 1033}
traj list st

In [6]:
kmc_boltz.plot_top_n_states_stacked(
    n_show = -1,
    max_time=0.0001, 
    title="State Distribution, {}K".format(T_kelvin),
    xlabel="Time (s)",
    ylabel="Population Proportion", 
    save=True, 
    show=True,
    save_name="./plots/Pt4H2_stacked_{}_top{}.png".format(T_kelvin, -1)
)

In [7]:
kmc_boltz.results_mat[:,0]

array([3036.00, 1952.00, 2024.00, 0.00, 0.00, 5.00, 0.00, 1037.00,
       1946.00, 0.00])

In [10]:
from bisect import bisect_left

bisect_left([1, 3, 5], 0)

0